In [21]:
import random
import os
from random import randint
import math
from IPython.display import display, HTML
import re
import numpy
def show_html(src):
    return display(HTML(src), metadata=dict(isolated=True))

## read svg code from files
- note: pixelated font don't have polyline so they don't work with this system :((((

In [34]:
char1name = '0502_楼熊_'
char2name = '0502_熊楼_'


mainDir = "svg text char pairs/" # main directory
with open(mainDir+f"{char1name}.svg", "r") as f1:
    char1 = list(f1)
with open(mainDir+f"{char2name}.svg", "r") as f2:
    char2 = list(f2)
# print(char2)

def index_containing_str(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

## get the specific segments svg code of blended character from each svg file n clean up lines

In [35]:
blendLayerTagS ='<g fill="none" id="layer2" inkscape:groupmode="layer" inkscape:label="2"'
blendLayerTagE = '</g>'

def getSVGcode(charAllLines, startLine, endLine):
    blendChar = []
    for begin in charAllLines:
        if startLine in begin: # get the index of the beginning line of the blended text svg code
            chSIndex = charAllLines.index(begin)
            chrEIndex = chSIndex + index_containing_str(charAllLines[chSIndex:len(charAllLines)], endLine) # get end index to this segment
            tempChar = charAllLines[chSIndex : chrEIndex] # svg code of the blended char, no svg tag
    
            for line in tempChar:
                nline = line.replace('\n', '')
                nnline = nline.replace('\t', '') # get rid of \n and \t
                if nnline[0].isdigit(): # merge all segments of a polyline tag line as one
                    blendChar[-1] += nnline
                else:
                    blendChar.append(nnline)
            return blendChar

def cleanUPfromList(SVGlist):  
    cleanedStr = ''
    for item in SVGlist:
        if '<g' in item:
            cleanedStr += '<g>'
        else:
            ridn = item.replace('\n', '')
            ridt = ridn.replace('\t', '')
            cleanedStr += ridt
#     print(cleanedStr)
    return cleanedStr
    
char1SVG = getSVGcode(char1, f'{blendLayerTagS}', f'{blendLayerTagE}')
Ori1SVG = getSVGcode(char1, '<g fill="none" id="layer1" inkscape:groupmode="layer" inkscape:label="1" stroke="#0000ff" style="display:inline">', f'{blendLayerTagE}')
Ori2SVG = getSVGcode(char1, '<g fill="none" id="layer3" inkscape:groupmode="layer" inkscape:label="3" stroke="#ff0000" style="display:inline">', f'{blendLayerTagE}')

char2SVG = getSVGcode(char2, f'{blendLayerTagS}', f'{blendLayerTagE}')


Ori1Paths  = cleanUPfromList(Ori1SVG)
Ori2Paths  = cleanUPfromList(Ori2SVG)

# print(char2SVG, len(char1SVG))
print(Ori1Paths)


<g>    <polyline points="188.9764,245.0787 184.8785,245.5368 180.9769,245.9785 177.2717,246.4038 173.7627,246.8128 170.4501,247.2054 167.3337,247.5816 164.4137,247.9415 161.69,248.2851 159.1625,248.6122 156.8314,248.923 154.6966,249.2175 152.7581,249.4956 151.0159,249.7573 149.47,250.0027 148.1204,250.2317 146.9671,250.4444 146.0101,250.6407 145.2494,250.8207 144.685,250.9843"/>    <line x1="144.685" x2="124.0157" y1="250.9843" y2="234.7441"/>    <polyline points="124.0157,234.7441 126.4287,234.8831 129.0052,234.9895 131.7453,235.0631 134.6489,235.104 137.7162,235.1122 140.9471,235.0876 144.3415,235.0304 147.8995,234.9404 151.6212,234.8177 155.5064,234.6623 159.5552,234.4742 163.7675,234.2533 168.1435,233.9998 172.6831,233.7135 177.3862,233.3945 182.2529,233.0428 187.2832,232.6584 192.4772,232.2412 197.8346,231.7913"/>    <polyline points="197.8346,231.7913 198.6035,230.6994 199.356,229.5993 200.0922,228.491 200.8119,227.3745 201.5154,226.2498 202.2024,225.117 202.8731,223.9759 203.527

## formatting polylines and lines data to svg structure... <s>the order of each tag doesn't seem to matter </s>    
actually the order matters very much :')...  <s>how to weave 2 lists together without a fixed pattern?? </s>


In [36]:
# modifying code
plineKey = '<polyline points="'
lineKey = '<line '




eachPairedPline = """
    <polyline class="{className}" stroke="white" fill="none">
        <animate attributeName="points" dur="9s" repeatCount="indefinite" 
        calcMode="spline" keyTimes="0; 0.5; 1"
        keySplines="0.5 0 0.5 1; 0.5 0 0.5 1"
         values="{plFromPath};{plToPath};{plFromPath}"
         </animate>
     </polyline>     
"""
eachStillPLine = """
    <polyline class="{className}"  points="{singlePline}">
    <animate attributeName="opacity" repeatCount="indefinite" 
    calcMode="spline" keyTimes="0; 0.5; 1"
    keySplines="0.5 0 0.5 1; 0.5 0 0.5 1"
    values="0.99;0.00;0.99" dur="9s">
    </animate>
    </polyline> 
"""



def sortComponents(toCharSVG, fromCharSVG, plinekey, linekey):    
    frompathiter = iter(fromCharSVG)

    for line in range(len(toCharSVG)): # based on the structure of targeted char svg list
        if plinekey in toCharSVG[line]: # if this is a polyline, reformat it at the same spot

            nline = toCharSVG[line][len(plinekey)+4:] #get rid of the prefix
            topath = nline[:-3] # get rid fo the suffix

            isMatchFromPlPath = next(frompathiter, "end") # loop through the fromChar plines
            mergedPLines=''
            if plinekey in isMatchFromPlPath:
                def genMergePL(isMatchFromPlPath, mergedPLines, plinekey, topath):
                    if plinekey in isMatchFromPlPath:
                        nline = isMatchFromPlPath[len(plinekey)+4:]
                        frompath = nline[:-3]

                        mergedPlDict={
                            'className': 'st2',
#                             'plDur': '5s',
#                             'plAnimFill': '',
                            'plFromPath': frompath,
                            'plToPath': topath,
                        }
                        newPline = eachPairedPline.format(**mergedPlDict)
                        mergedPLines += newPline
                        genMergePL(next(frompathiter, "end"), mergedPLines, plinekey, topath)
                    return mergedPLines
                toCharSVG[line] = genMergePL(isMatchFromPlPath, mergedPLines, plineKey, topath)
                print(toCharSVG[line])

            else:
                stillPLDict={
                    'className': 'st2',
                    'singlePline': topath,
                }
                toCharSVG[line] = eachStillPLine.format(**stillPLDict)

#             print(line, toCharSVG[line])
            
#                 else:
#                     stillPLDict={
#                         'className': 'st2',
#                         'singlePline': topath,
#                     }
#                     singlePLine = eachStillPLine.format(**stillPLDict)
#                     return singlePLine
                
#             toCharSVG[line] = genMergePL(isMatchFromPlPath, mergedPLines, plinekey, topath)
#             print(line, toCharSVG[line])

        elif linekey in toCharSVG[line]:
            def genMergeL(isMatchFromPlPath):   
                if linekey in isMatchFromPlPath:
                    toCharSVG[line] += isMatchFromPlPath
                    genMergeL(next(frompathiter, "end"))
                else:
                    return toCharSVG[line]
            
            isMatchFromPlPath = next(frompathiter, "end")
#             print(line, toCharSVG[line])

        
        else:
            continue
    
                 
#     print(toCharSVG)        
    return toCharSVG

pathslist = sortComponents(char2SVG, char1SVG, plineKey, lineKey)
mergedPaths = cleanUPfromList(pathslist)

# print(mergedPaths)




    <polyline class="st2" stroke="white" fill="none">
        <animate attributeName="points" dur="9s" repeatCount="indefinite" 
        calcMode="spline" keyTimes="0; 0.5; 1"
        keySplines="0.5 0 0.5 1; 0.5 0 0.5 1"
         values="200.7874,134.3504 194.8982,139.393 189.3526,143.953 184.1505,148.0304 179.292,151.6252 174.777,154.7375 170.6055,157.3672 166.7775,159.5143 163.2931,161.1788 160.1522,162.3607 157.3549,163.06 154.9011,163.2768 152.7908,163.0109 151.0241,162.2625 149.6008,161.0315 148.5212,159.3179 147.785,157.1218 147.3924,154.443 147.3433,151.2817 147.6378,147.6378;138.7795,246.5551 147.8832,242.8499 156.2343,239.8153 163.833,237.4515 170.6791,235.7583 176.7727,234.7359 182.1139,234.3842 186.7025,234.7032 190.5386,235.6929 193.6223,237.3533 195.9534,239.6844 197.532,242.6863 198.3581,246.3588 198.4317,250.7021 197.7529,255.716 196.3215,261.4007 194.1376,267.7561 191.2012,274.7822 187.5123,282.479 183.0709,290.8465;200.7874,134.3504 194.8982,139.393 189.3526,143.953 

<br>
<br>
<br>
<br>

## format original character forms

In [37]:

def formatOriChar(charstr, classname):
    formatted = ''
    oriCharList = charstr.split('    ')
    for line in range(len(oriCharList)):
        if '<polyline' in oriCharList[line]:
            newline = oriCharList[line][:len('<polyline ')] + f'class="{classname}"' + oriCharList[line][len('<polyline'):] 
#             oriCharList[line] = newline
            formatted+=newline

        if '<line' in oriCharList[line]:
            newline = oriCharList[line][:len('<line ')] + f'class="{classname}"' + oriCharList[line][len('<line'):] 
#             oriCharList[line] = newline
            formatted+=newline

    return formatted

Ori1PathsFormated = formatOriChar(Ori1Paths, 'st1')
Ori2PathsFormated = formatOriChar(Ori2Paths, 'st3')

# print(Ori2PathsFormated)




In [38]:
# original code ignoring tag orders... don't use this

# plinekey = f'<polyline class="st2" points="'
# linekey = f'<line'


# def sortComponents(charSVG, plinekey, linekey):
#     plines = []
#     lines = []
#     for line in charSVG:
#         if plinekey in line:
#             plines.append(line)
#         if linekey in line:
#             lines.append(line)
#     return plines, lines

# char1Plines, char1Lines = sortComponents(char1SVG, plinekey, linekey)
# char2Plines, char2Lines = sortComponents(char2SVG, plinekey, linekey)
# Ori1Plines, Ori1Lines = sortComponents(char1SVG, '<polyline class="st1" points="', linekey)
# Ori2Plines, Ori2Lines = sortComponents(char1SVG, '<polyline class="st3" points="', linekey)


# def getPlinePaths(charPlines, plinekey):
#     plinepaths=[]
#     for line in charPlines:
#         nline = line[len(plinekey)+1:] #get rid of the prefix
#         nnline = nline[:-3] # get rid fo the suffix
#         plinepaths.append(nnline)
#     return plinepaths

# char1PlinePaths = getPlinePaths(char1Plines, plinekey)
# char2PlinePaths = getPlinePaths(char2Plines, plinekey)
# Ori1PlinesPaths = getPlinePaths(char1Plines, '<polyline class="st1" points="')
# Ori2PlinesPaths = getPlinePaths(char1Plines, '<polyline class="st1" points="')

# # print(len(char1PlinePaths), len(char2PlinePaths))
# # print(Ori1PlinesPaths)

## generate code per polyline tag for html and animation
    # each pline from char1 is paired up with each plien from char2; base: the char with less pliens
### then gather all svg code

In [39]:
# following the original code ignoring tag orders... don't use this either

# toChar = char1PlinePaths
# fromChar = char2PlinePaths
# allPlines = ''

# eachPairedPline = """
#     <polyline class="{className}">
#         <animate 
#         attributeName="points" dur="{plDur}" fill="{plAnimFill}" repeatCount="indefinite"
#          from="{plFromPath}"
#          to="{plToPath}
#          "/>         
#      </polyline>
     
# """

# eachStillPline = """
#     <polyline class="{className}" 
#     attributeName="stop-opacity" values="0.99;0.00;0.99" dur="3s" repeatCount="indefinite" />
#     points="
#     {singlePline}
#     " >
#     </polyline>
# """

# for i in range(len(fromChar)):
#     plDict={
#         'className': 'st2',
#         'plDur': '4s',
#         'plAnimFill': 'freeze',
#         'plFromPath': fromChar[i],
#         'plToPath': toChar[i],
#     }
#     eachPlinePath = eachPairedPline.format(**plDict)
#     allPlines += eachPlinePath

# for line in char1PlinePaths[len(char2PlinePaths):]:
#     stillplDict={
#         'className': 'st2',
#         'singlePline': line,
#     }
#     eachStillPlinePath = eachStillPline.format(**stillplDict)
#     allPlines+=eachStillPlinePath
    
# # for line in char1Lines:
# #     allPlines += line
# # for line in char2Lines:
# #     allPlines += line




# Ori1Path = ''
# for line in Ori1PlinesPaths:
#     stillplDict={
#         'className': 'st1',
#         'singlePline': line,
#     }
#     Ori1Path += eachStillPline.format(**stillplDict)
# Ori2Path = ''
# for line in Ori2PlinesPaths:
#     stillplDict={
#         'className': 'st3',
#         'singlePline': line,
#     }
#     Ori2Path += eachStillPline.format(**stillplDict)


# print(len(allPlines))



In [40]:
html_template = """

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title> 字 </title>
    <link rel="stylesheet" href="../char.css">
    <style>
    html {{ background-color: grey; }}    
    </style>
</head>
<body>

    <svg class="ch1svg" viewBox="0 0 500 500">
        {OriChar1}
    </svg>

    <svg class="blendsvg" viewBox="0 0 500 500">
        {Plines}
    </svg>
    
    <svg class="ch2svg" viewBox="0 0 500 500">
    {OriChar2}
    </svg>


</body>
</html>


"""

In [41]:
htmlDict={
        'Plines': mergedPaths, # allPlines +Ori1Path + Ori2Path, 
        'OriChar1': Ori1PathsFormated,
        'OriChar2': Ori2PathsFormated,
    }

svgpage = html_template.format(**htmlDict)
open(f"pages/0502_累睡.html", "w").write(svgpage)

95439